<h1 style="font-size:3rem;color:orange:"> Effect of mask mandates on Airbnb occupancy rate


In [23]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import linearmodels as lm

# Fast load

In [24]:
hw_df=pd.read_csv('C:/Users/huyen/My Drive/UCR_Drive/Huyen Thao Le_project/Airbnb/hw_occupancyD.csv')
bs_df=pd.read_csv('C:/Users/huyen/My Drive/UCR_Drive/Huyen Thao Le_project/Airbnb/bs_occupancyD.csv')



In [25]:
hw_df.head()

,accommodates,bedrooms,price,distance,size,id,host_id,host_since,host_is_superhost,host_has_profile_pic,...,time,available,revenue,place,D,modest_occupancy,modest_income,Smoke_alarm,TV,Long_term_stays_allowed
0,2,1.0,150.0,35.080990,234.989048,5269,7620,12,f,t,...,202012,1,150.0,hw,0,0.85,127.50,True,True,True
1,2,1.0,85.0,34.742662,207.506314,5387,7878,12,f,t,...,202012,1,85.0,hw,0,11.94,1014.90,True,True,False
2,6,3.0,239.0,23.355417,10.000000,5389,7878,12,f,t,...,202012,1,239.0,hw,0,2.26,540.14,True,True,False
3,6,3.0,290.0,466.411506,231.531459,5434,7984,12,t,t,...,202012,1,290.0,hw,0,7.43,2154.70,True,True,False
4,4,1.0,183.0,129.482255,284.840544,7896,21844,12,f,t,...,202012,1,183.0,hw,0,2.16,395.28,True,True,True


In [26]:
hw_df.drop(['id','latitude',
       'longitude', 'property_type','room_type', 'availability_60','availability_90', 'availability_365'],axis=1, inplace=True)
bs_df.drop(['id','latitude',
       'longitude', 'property_type','room_type', 'availability_60','availability_90', 'availability_365'],axis=1, inplace=True)



In [30]:
def change_encode_feature(df0):
    df = df0.copy()
    for col in ['host_has_profile_pic','host_is_superhost','host_identity_verified','instant_bookable']:
        df[col] = df[col].astype('category')
    df.reset_index(drop=True,inplace=True)
    return df

In [32]:
def change_encode_feature(df0):
    df = df0.copy()
    for col in ['host_has_profile_pic','host_is_superhost','host_identity_verified','instant_bookable']:
        df[col] = df[col].astype('category')
    df.reset_index(drop=True,inplace=True)

    # target = df[['availability_30','availability_60', 'availability_365','calculated_host_listings_count','minimum_nights', 'price', 'modest_occupancy', 'modest_income']]

    features = df.copy()
    num_feats = features.select_dtypes(include=['float64', 'int64', 'bool']).copy()

    # one-hot encoding of categorical features
    cat_feats = features.select_dtypes(include=['category']).copy()
    cat_feats = pd.get_dummies(cat_feats, drop_first=True)
    features_recoded = pd.concat([num_feats, cat_feats], axis=1) 
    return features_recoded  
features_recoded_hw = change_encode_feature(hw_df)
features_recoded_bs = change_encode_feature(bs_df)

def aggregation(x):       
    df = x.groupby(['host_id','time'])['availability_30'].agg('mean').to_frame()
    for i in ['calculated_host_listings_count','minimum_nights','price','distance','host_since',\
              'modest_occupancy', 'modest_income', 'maximum_nights']:
        df[i] = x.groupby(['host_id','time'])[i].agg('mean').to_frame()[i]
    for i in ['accommodates','bedrooms','size','reviews_per_month']:
        df[i] = x.groupby(['host_id','time'])[i].agg('sum').to_frame()[i]
    for i in ['host_is_superhost_t','host_has_profile_pic_t','host_identity_verified_t',\
                'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms','Smoke_alarm', 'TV', 'Long_term_stays_allowed','instant_bookable_t']:
        df[i] = x.groupby(['host_id','time'])[i].agg('max').to_frame()[i]         
    
    return df

hw = aggregation(features_recoded_hw)
bs = aggregation(features_recoded_bs)

def scale_df(df):
    target = df[['availability_30','calculated_host_listings_count','minimum_nights',  'modest_occupancy', 'modest_income', 'price']]
 
    features = df.drop(['availability_30','calculated_host_listings_count','minimum_nights',\
                        'modest_income','modest_occupancy'], axis=1)
    cols=features.columns
    # scale data
    sc = StandardScaler()
    features_recoded = sc.fit_transform(features)
    features_recoded = pd.DataFrame(features_recoded, columns = cols)
    features_recoded.set_index(features.index,inplace=True)
    return target, features_recoded

target_hw, feature_hw = scale_df(hw)
target_bs, feature_bs = scale_df(bs)




# Prepare data

In [33]:
hw_assume = pd.concat([target_hw.drop(['price'], axis=1), feature_hw],axis=1)
# ny_assume = pd.concat([target_ny.drop(['price'], axis=1), feature_ny],axis=1)
bs_assume = pd.concat([target_bs.drop(['price'], axis=1), feature_bs],axis=1)
# fl_assume = pd.concat([target_fl.drop(['price'], axis=1), feature_fl],axis=1)

def prepareDiD(df_treatment, df_control, drop_time, time_policy, lag=False):
    df_treatment0=df_treatment.copy()
    df_control0=df_control.copy()
    df_treatment0['D']=1
    df_control0['D']=0
    df = pd.concat([df_treatment0, df_control0], axis=0)

    time = df.index.get_level_values(1)

    df.drop(index=df[time>=drop_time].index, inplace=True)
    time = df.index.get_level_values(1)
    # df.drop(index=df[time==time_policy].index, inplace=True)
    # time = df.index.get_level_values(1)

    if lag:
        df['T'] = [1 if x > time_policy else 0 for x in time]
    else:
        df['T'] = [1 if x >=time_policy else 0 for x in time]
    df['D*T'] = df['D']*df['T']  
    # df.drop(columns=['D', 'T'], inplace=True)    
    # df['host_has_profile_pic']=[1 if x == 't' else 0 for x in df['host_has_profile_pic']]
    # df['host_is_superhost']=[1 if x == 't' else 0 for x in df['host_is_superhost']]
    # df['host_identity_verified']=[1 if x == 't' else 0 for x in df['host_identity_verified']]
    df.index = df.index.set_levels(df.index.levels[1].astype(int), level=1)
    df.dropna(inplace=True)
    return df

In [34]:
hw_assume1 = hw_assume.drop(hw_assume[hw_assume.index.get_level_values(1)<202011].index)
df_feb_ny = prepareDiD(bs_assume, hw_assume1, 202105, 202103)
df_feb_ny.head(10)

availability_30  calculated_host_listings_count  \
host_id time                                                      
416912  202012             30.0                             1.0   
        202101              0.0                             1.0   
        202102             27.0                             1.0   
        202103             30.0                             1.0   
        202104             30.0                             1.0   
480310  202103              1.0                             1.0   
        202104              2.0                             1.0   
494453  202012              0.0                             1.0   
        202101              0.0                             1.0   
        202102              0.0                             1.0   

                minimum_nights  modest_occupancy  modest_income     price  \
host_id time                                                                
416912  202012             2.0              3.74         523.60  0.145238   
        202101             2.0              3.67         513.80  0.145238   
        202102             2.0              3.67         513.80  0.145238   
        202103             2.0              3.67         513.80  0.145238   
        202104             2.0              3.59         502.60  0.145238   
480310  202103            29.0             11.93        2934.78  1.664967   
        202104            29.0             11.86        2312.70  0.933777   
494453  202012             1.0             15.29        2522.85  0.503665   
        202101             1.0             14.90        2458.50  0.503665   
        202102             1.0             14.51        2394.15  0.503665   

                distance  host_since  maximum_nights  accommodates  ...  \
host_id time                                                        ...   
416912  202012 -1.000045    1.964888       -0.078969     -0.481538  ...   
        202101 -1.000045    1.964888       -0.078969     -0.481538  ...   
        202102 -1.000045    1.964888       -0.078969     -0.481538  ...   
        202103 -1.000045    1.964888       -0.078969     -0.481538  ...   
        202104 -0.983612    1.964888       -0.078969     -0.481538  ...   
480310  202103 -1.121062    1.964888        0.735809      0.050683  ...   
        202104 -1.121062    1.964888        0.735809      0.050683  ...   
494453  202012 -0.241668    1.964888        0.737877     -0.162206  ...   
        202101 -0.241668    1.964888        0.737877     -0.162206  ...   
        202102 -0.241668    1.964888        0.737877     -0.162206  ...   

                calculated_host_listings_count_entire_homes  \
host_id time                                                  
416912  202012                                    -0.409983   
        202101                                    -0.409983   
        202102                                    -0.409983   
        202103                                    -0.409983   
        202104                                    -0.409983   
480310  202103                                    -0.137102   
        202104                                    -0.137102   
494453  202012                                    -0.137102   
        202101                                    -0.137102   
        202102                                    -0.137102   

                calculated_host_listings_count_private_rooms  \
host_id time                                                   
416912  202012                                     -0.003875   
        202101                                     -0.003875   
        202102                                     -0.003875   
        202103                                     -0.003875   
        202104                                     -0.003875   
480310  202103                                     -0.420574   
        202104                                     -0.420574   
494453  202012                                     -

# DID


## Placebo test for parallel trend assumption

In [36]:
hw_assume1 = hw_assume.drop(hw_assume[hw_assume.index.get_level_values(1)<202011].index)
df_feb_ny = prepareDiD(bs_assume, hw_assume1, 202105, 202103)
feature_hw_fe=df_feb_ny.drop(['availability_30','calculated_host_listings_count','minimum_nights', 'modest_occupancy', 'modest_income'], axis=1)
target_hw_fe=df_feb_ny[['availability_30','calculated_host_listings_count','minimum_nights','modest_occupancy', 'modest_income']]

for outcome_ in ['availability_30','calculated_host_listings_count','modest_occupancy', 'modest_income']:
    print('\n%s'%outcome_)
    exog = sm.add_constant(feature_hw_fe)
    endog = target_hw_fe[outcome_]

    model_lr = sm.OLS(endog, exog) 
    lr_res = model_lr.fit() 
    #print results

    print(lr_res.summary())
    print()


availability_30
                            OLS Regression Results                            
Dep. Variable:        availability_30   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     140.2
Date:                Mon, 17 Oct 2022   Prob (F-statistic):               0.00
Time:                        11:38:22   Log-Likelihood:            -1.1534e+05
No. Observations:               32133   AIC:                         2.307e+05
Df Residuals:                   32111   BIC:                         2.309e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

# Effect of mask mandates on listings and occupancy rates


In [38]:
hw_assume1 = hw_assume.drop(hw_assume[hw_assume.index.get_level_values(1)<202011].index)
df_feb_ny = prepareDiD(bs_assume, hw_assume1, 202112, 202105, lag=False)
feature_hw_fe=df_feb_ny.drop(['availability_30','calculated_host_listings_count','minimum_nights', 'modest_occupancy', 'modest_income'], axis=1)
target_hw_fe=df_feb_ny[['calculated_host_listings_count','minimum_nights','modest_occupancy', 'modest_income']]

for outcome_ in ['calculated_host_listings_count','modest_occupancy', 'modest_income']:
    print('\n%s'%outcome_)
    exog = sm.add_constant(feature_hw_fe)
    endog = target_hw_fe[outcome_]

    model_lr = sm.OLS(endog, exog) 
    lr_res = model_lr.fit() 
    #print results

    print(lr_res.summary())
    print()


calculated_host_listings_count
                                  OLS Regression Results                                  
Dep. Variable:     calculated_host_listings_count   R-squared:                       0.944
Model:                                        OLS   Adj. R-squared:                  0.944
Method:                             Least Squares   F-statistic:                 5.443e+04
Date:                            Mon, 17 Oct 2022   Prob (F-statistic):               0.00
Time:                                    11:40:42   Log-Likelihood:            -1.5784e+05
No. Observations:                           67385   AIC:                         3.157e+05
Df Residuals:                               67363   BIC:                         3.159e+05
Df Model:                                      21                                         
Covariance Type:                        nonrobust                                         
                                                   coef   